In [2]:
!pip install opencv-python
!pip install scikit-image
!pip install numpy

In [3]:
import cv2
from skimage import measure, morphology
import numpy as np

In [14]:
def signature_detection(image:np.ndarray,output=None) -> None:
    # Gray Scale Conversion 
    img=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # smoothing
    # img=cv2.bilateralFilter(img,35, 105, 105)
    # Binary Scale Conversion
    img = cv2.threshold(img, 115, 255, cv2.THRESH_BINARY)[1] 
    # Connected Reagion Detection   
    blobs_labels = measure.label((img > img.mean()), background=1)
    # Mean Area calculation
    mean_area = 0; counter = 0   
    for region in measure.regionprops(blobs_labels):
        if (region.area > 10):
            mean_area = mean_area + region.area
            counter = counter + 1
    mean_area = (mean_area/counter)
    # Min_Max_outliar calculation
    min_outliar = (mean_area*2.9762)+100
    max_outliar = min_outliar*18    
    # Removing Outliar 
    img = morphology.remove_small_objects(blobs_labels, min_outliar) 
    small_mask = np.bincount(img.reshape(-1)) > max_outliar  
    img[small_mask[img]] = 0
    # Normalization
    img = np.array((img/img.max())*255,np.uint8)
    # Binary image Conversion
   
    thresh = 100+(img.mean()**2)*(100/255)*2

    img = cv2.threshold(img, thresh, 255, cv2.THRESH_BINARY_INV)[1]
    # Erode image to find signature
    img = cv2.erode(img, np.ones((21,21)),50)
    # Contour detection
    contour = cv2.findContours(img,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)[0]    
    c = sorted(contour,key =cv2.contourArea)[-2]
    # Boundary Drawing    
    x,y,w,h =cv2.boundingRect(c)
    img = cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),3)
    # disp(img)
    cv2.imwrite(output, img)


In [11]:
image_dir = '../images/signatureDocuments'
result_dir= '../result'

In [6]:
import os 
l=[]
for i in os.listdir(image_dir):
    l+=[i]
k=0
for i in l:    
    try:
        img = cv2.imread(os.path.join(image_dir,i))
        signature_detection(img, f"{result_dir}/{i}")
    except:
        pass
    

/tmp/ipykernel_49215/2272441338.py:25: RuntimeWarning: invalid value encountered in divide
  img = np.array((img/img.max())*255,np.uint8)
/tmp/ipykernel_49215/2272441338.py:25: RuntimeWarning: invalid value encountered in cast
  img = np.array((img/img.max())*255,np.uint8)


In [12]:
img = cv2.imread("/home/aritrarc/Downloads/4.jpeg")
signature_detection(img, f"{result_dir}/{i}")